In [1]:
from utils import get_webpage_content, extract_params, process_model_output, filter_json
import json
from json5 import loads
import pandas as pd
pd.set_option('display.max_colwidth', None)
from tqdm import tqdm

In [ ]:
example_single = "I need beers that have an EBC less than 20, brewed after 01-2025, with an ABV of more than 5%, and yeast named Brettanomyces."
processed_text = extract_params(example_single)
print('*' + processed_text)
processed_text = process_model_output(processed_text)
print('**' + processed_text)
processed_text = loads(processed_text)
print(str(processed_text))
print('-----------------')
processed_text_filtered, errors = filter_json(processed_text)
print(str(processed_text_filtered), errors)

In [2]:
with open('test_cases.txt', 'r') as f:
    examples = f.read().split('\n')

examples_processed = [example.split("\"")[1].split("\"")[0] for example in examples]
examples_cases = [example.split("(")[1].split(")")[0] if len(example.split("(")) > 1 else None for example in examples]

In [3]:
examples_list = []
case_list = []
output_list = []
output_filtered_list = []
error_list = []

for i in tqdm(range(10)):
    examples_list.append(examples_processed[i])
    case_list.append(examples_cases[i])
    count = 0
    while count < 2:
        try:
            processed_text = loads(process_model_output(extract_params(examples_processed[i])))
            processed_text_filtered, errors = filter_json(processed_text)
            output_list.append(processed_text)
            output_filtered_list.append(processed_text_filtered)
            error_list.append(errors)
            count = 2
        except:
            count += 1
            if count == 2:
                output_list.append("{}")
                output_filtered_list.append("{}")
                error_list.append("")
            continue

100%|██████████| 10/10 [00:19<00:00,  1.99s/it]


In [4]:
pd.DataFrame({"Example": examples_list, "Edge_Case": case_list, "Output": output_list, "Filtered_Output": output_filtered_list, "Errors": error_list})

,Example,Edge_Case,Output,Filtered_Output,Errors
0,Find a beer with ABV greater than 8%.,None,{'abv_gt': 8},{'abv_gt': 8},
1,I want a beer brewed before 10-2010.,None,{'brewed_before': '10-2010'},{'brewed_before': '10-2010'},
2,I'm looking for beers with ABV less than 5% and brewed after 01-2015.,None,"{'abv_lt': 5, 'brewed_after': '01-2015'}","{'abv_lt': 5, 'brewed_after': '01-2015'}",
3,Give me a beer that contains the yeast Saccharomyces Pastorianus.,None,{'yeast': 'Saccharomyces Pastorianus'},{},
4,I want a beer with IBU greater than 40 and malt Maris Otter Extra Pale.,None,"{'ibu_gt': 40, 'malt': 'Maris Otter Extra Pale'}",{'ibu_gt': 40},
5,Find a beer brewed after 01-2012 with ABV less than 5%.,None,"{'abv_lt': 5, 'brewed_after': '01-2012'}","{'abv_lt': 5, 'brewed_after': '01-2012'}",
6,"I want a beer with ABV greater than 6%, brewed before 05-2015, and made with Simcoe hops.",None,"{'abv_gt': 6, 'brewed_before': '05-2015', 'hops': 'Simcoe'}","{'abv_gt': 6, 'brewed_before': '05-2015'}",
7,Find a beer that matches the name Punk IPA.,None,{'beer_name': 'Punk IPA'},{},
8,Give me a beer brewed after 01-2015 with yeast Brettanomyces.,None,"{'brewed_after': '01-2015', 'yeast': 'Brettanomyces'}",{'brewed_after': '01-2015'},
9,Find a beer that pairs well with beef stew.,None,{'food': 'beef stew'},{},


In [35]:
query_string = "&".join([f"{key}={value}" for key, value in output_filtered_list[7].items()] + [f"per_page={80}"])

In [36]:
URL = "https://api.punkapi.com/v2/beers?" + query_string + "&page=1"
URL

'https://api.punkapi.com/v2/beers?per_page=80&page=1'

In [37]:
api_call_list = []
counter = 1
state = True
while state:
    try:
        content = get_webpage_content("https://api.punkapi.com/v2/beers?" + query_string + f"&page={counter}")
        content = loads(content)
        if content == []:
            state = False
            break
        api_call_list.append(content)
        counter += 1
        
    except Exception as e:
        print(e)
        break

api_call_list_flattened = [item for sublist in api_call_list for item in sublist]
len(api_call_list_flattened)

325

In [34]:
api_call_list_flattened

[{'id': 3,
  'name': 'Berliner Weisse With Yuzu - B-Sides',
  'tagline': 'Japanese Citrus Berliner Weisse.',
  'first_brewed': '11/2015',
  'description': 'Japanese citrus fruit intensifies the sour nature of this German classic.',
  'image_url': 'https://images.punkapi.com/v2/keg.png',
  'abv': 4.2,
  'ibu': 8,
  'target_fg': 1007,
  'target_og': 1040,
  'ebc': 8,
  'srm': 4,
  'ph': 3.2,
  'attenuation_level': 83,
  'volume': {'value': 20, 'unit': 'litres'},
  'boil_volume': {'value': 25, 'unit': 'litres'},
  'method': {'mash_temp': [{'temp': {'value': 60, 'unit': 'celsius'},
     'duration': 10},
    {'temp': {'value': 65, 'unit': 'celsius'}, 'duration': 30},
    {'temp': {'value': 72, 'unit': 'celsius'}, 'duration': 10},
    {'temp': {'value': 78, 'unit': 'celsius'}, 'duration': 5}],
   'fermentation': {'temp': {'value': 21, 'unit': 'celsius'}},
   'twist': 'Soured naturally using the kettle souring technique, Yuzu fruit: 50g at middle, Yuzu juice: 200ml at FV'},
  'ingredients': {

In [38]:
sample = output_list[7]
keys_to_keep = ["beer_name", "yeast", "hops", "malt", "food"]
sample = [sample[key] for key in sample if key in keys_to_keep]
sample


['Punk IPA']

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

sample = output_list[7]
keys_to_keep = ["beer_name", "yeast", "hops", "malt", "food"]
sample = [sample[key] for key in sample if key in keys_to_keep]
sample

beer_descriptions = [str(description) for description in api_call_list_flattened]
keywords = sample

# join keywords into a single string
keywords_string = ' '.join(keywords)

# add it to the beer descriptions
texts = beer_descriptions + [keywords_string]

# initialize TfidfVectorizer
vectorizer = TfidfVectorizer()

# fit and transform the vectorizer on texts
tfidf_matrix = vectorizer.fit_transform(texts)

# calculate cosine similarity
cosine_similarities = cosine_similarity(tfidf_matrix[-1:], tfidf_matrix[:-1])

# print beers with their similarity scores
beer_list = []
score_list = []
for i, score in enumerate(cosine_similarities[0]):
    beer_list.append(api_call_list_flattened[i])
    score_list.append(score)


In [43]:
pd.DataFrame({"Beer": beer_list, "Score": score_list}).sort_values(by="Score", ascending=False).head(3)

,Beer,Score
94,"{'id': 95, 'name': 'Peroxide Punk', 'tagline': 'Zesty Pale Ale.', 'first_brewed': '04/2007', 'description': 'A trashy blonde concession for those who felt that our original 6% Punk IPA recipe was too hard hitting. This was also the first time we experimented with dry hopping our beers, giving Peroxide Punk a depth of flavour that belies its modest ABV. Zesty, aromatic and thirst quenching.', 'image_url': 'https://images.punkapi.com/v2/keg.png', 'abv': 4, 'ibu': 40, 'target_fg': 1009, 'target_og': 1039, 'ebc': 18, 'srm': 9, 'ph': 4.4, 'attenuation_level': 76.9, 'volume': {'value': 20, 'unit': 'litres'}, 'boil_volume': {'value': 25, 'unit': 'litres'}, 'method': {'mash_temp': [{'temp': {'value': 65, 'unit': 'celsius'}, 'duration': 75}], 'fermentation': {'temp': {'value': 19, 'unit': 'celsius'}}, 'twist': None}, 'ingredients': {'malt': [{'name': 'Maris Otter', 'amount': {'value': 3.85, 'unit': 'kilograms'}}], 'hops': [{'name': 'Amarillo', 'amount': {'value': 25, 'unit': 'grams'}, 'add': 'start', 'attribute': 'bitter'}, {'name': 'Simcoe', 'amount': {'value': 20, 'unit': 'grams'}, 'add': 'end', 'attribute': 'flavour'}, {'name': 'Amarillo', 'amount': {'value': 25, 'unit': 'grams'}, 'add': 'end', 'attribute': 'flavour'}, {'name': 'Crystal', 'amount': {'value': 20, 'unit': 'grams'}, 'add': 'end', 'attribute': 'flavour'}, {'name': 'Liberty', 'amount': {'value': 20, 'unit': 'grams'}, 'add': 'end', 'attribute': 'flavour'}, {'name': 'Simcoe', 'amount': {'value': 50, 'unit': 'grams'}, 'add': 'dry hop', 'attribute': 'aroma'}, {'name': 'Amarillo', 'amount': {'value': 50, 'unit': 'grams'}, 'add': 'dry hop', 'attribute': 'aroma'}], 'yeast': 'Wyeast 1056 - American Ale™'}, 'food_pairing': ['Sashimi', 'Pan roasted poussin with spring veg', 'Lemon and thyme panna cotta'], 'brewers_tips': 'During the mashing process, try to mix the malt without 'turning' or splashing too much. This will cause more oxygen to be dissolved in the mash - and result in a beer that will taste a little like a cardboard box.', 'contributed_by': 'Sam Mason '}",0.232938
191,"{'id': 192, 'name': 'Punk IPA 2007 - 2010', 'tagline': 'Post Modern Classic. Spiky. Tropical. Hoppy.', 'first_brewed': '04/2007', 'description': 'Our flagship beer that kick started the craft beer revolution. This is James and Martin's original take on an American IPA, subverted with punchy New Zealand hops. Layered with new world hops to create an all-out riot of grapefruit, pineapple and lychee before a spiky, mouth-puckering bitter finish.', 'image_url': 'https://images.punkapi.com/v2/192.png', 'abv': 6, 'ibu': 60, 'target_fg': 1010, 'target_og': 1056, 'ebc': 17, 'srm': 8.5, 'ph': 4.4, 'attenuation_level': 82.14, 'volume': {'value': 20, 'unit': 'litres'}, 'boil_volume': {'value': 25, 'unit': 'litres'}, 'method': {'mash_temp': [{'temp': {'value': 65, 'unit': 'celsius'}, 'duration': 75}], 'fermentation': {'temp': {'value': 19, 'unit': 'celsius'}}, 'twist': None}, 'ingredients': {'malt': [{'name': 'Extra Pale', 'amount': {'value': 5.3, 'unit': 'kilograms'}}], 'hops': [{'name': 'Ahtanum', 'amount': {'value': 17.5, 'unit': 'grams'}, 'add': 'start', 'attribute': 'bitter'}, {'name': 'Chinook', 'amount': {'value': 15, 'unit': 'grams'}, 'add': 'start', 'attribute': 'bitter'}, {'name': 'Crystal', 'amount': {'value': 17.5, 'unit': 'grams'}, 'add': 'middle', 'attribute': 'flavour'}, {'name': 'Chinook', 'amount': {'value': 17.5, 'unit': 'grams'}, 'add': 'middle', 'attribute': 'flavour'}, {'name': 'Ahtanum', 'amount': {'value': 17.5, 'unit': 'grams'}, 'add': 'end', 'attribute': 'flavour'}, {'name': 'Chinook', 'amount': {'value': 27.5, 'unit': 'grams'}, 'add': 'end', 'attribute': 'flavour'}, {'name': 'Crystal', 'amount': {'value': 17.5, 'unit': 'grams'}, 'add': 'end', 'attribute': 'flavour'}, {'name': 'Motueka', 'amount': {'value': 17.5, 'unit': 'grams'}, 'add': 'end', 'attribute': 'flavour'}], 'yeast': 'Wyeast 1056 - American Ale™'}, 'food_pairing': ['Spicy carne asada with a pico de gallo sauce', '